### Import dependencies

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install squarify
#plt.style.use('fivethirtyeight')
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import pygsheets
%matplotlib inline


### Authorize connection between Jupyter notebook (python working environment; like R studio) and google sheets so we can run analyses as the sheet is updated

In [2]:
gc = pygsheets.authorize() 
# Use customized credentials 
gc = pygsheets.authorize(client_secret='client_secret.json')
# For the first time, it will may produce as a link to authorize

### Open spreadsheet by name

In [3]:
sh = gc.open('PythonCurrentFlow_Aging')

### Open "Data" worksheet (can also use sh.sheet1)

In [4]:
data = sh[0]

### Get worksheet values as pandas dataframe

In [8]:
aging_data = pd.DataFrame(data.get_all_records())

In [9]:
aging_data

,Genus,Species,Sample ID,Species Code,Site,River,Basin,Lat,Long,Date Collected,...,Status Upon Collection,Final Age,Age dif,Z age,L age,B age,K&M Age,K age,M age,Notes
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,15,14,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,16,,,,,16,16,12,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,17,,,,,17,17,9,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,13,12,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,14,,,,,14,14,12,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,B: how could you tell difference b/w 0.5 & 1?


### What do age measurements look like across Z, L & B?
#### Compare value counts 

In [23]:
pd.crosstab(index=aging_data['Species'], columns=aging_data['Z age'])

Z age,0.5,1,2,3,4,5,6,7,8,9,12,13,,NA
Species,,,,,,,,,,,,,,
plicata,0,1,1,0,0,0,0,0,1,0,1,1,320,0
teres,7,6,12,33,22,18,14,10,7,2,0,0,16,2


##### L. teres
#### Ager Z: aged most mussels at 3 yrs old

In [24]:
pd.crosstab(index=aging_data['Species'], columns=aging_data['L age'])

L age,0.5,1,2,3,4,5,6,7,8,9,10,11,13,,broken
Species,,,,,,,,,,,,,,,
plicata,0,1,1,0,0,0,0,0,0,1,1,0,1,320,0
teres,7,0,6,19,26,34,17,8,10,1,2,1,0,15,3


##### L. teres
#### Ager L: aged most mussels at 5 yrs old

In [25]:
pd.crosstab(index=aging_data['Species'], columns=aging_data['B age'])

B age,1,2,3,4,5,6,7,8,9,,,
Species,,,,,,,,,,,,
plicata,0,0,0,0,0,0,0,0,0,325,0,0
teres,9,12,25,27,21,12,10,5,2,24,1,1


##### L. teres
#### Ager B: aged most mussels at 4 yrs old

In [35]:
pd.crosstab(index=aging_data['Species'], columns=aging_data['Site'])

Site,"Altair, TX","Bay City, TX","Bellville, TX","Gonzales, TX","Navasota, TX","Simonton, TX","Victoria, TX"
Species,,,,,,,
plicata,100,0,5,101,19,0,100
teres,13,67,19,0,0,50,0


### Clean and transform data
#### Unpivot agers (Z, L & B) aka 'wide format' to 'long format' 

In [40]:
aging_melt=aging_data.melt(id_vars=['Sample ID', 'Genus', 'Species', 'Species Code', 'Site', 'River', 'Basin', 'Lat', 'Long', 'Date Collected', 'Status Upon Collection'],
                value_vars=["Z age", "L age", "B age"],
                var_name="Ager",
                value_name="Age")
aging_melt               

,Sample ID,Genus,Species,Species Code,Site,River,Basin,Lat,Long,Date Collected,Status Upon Collection,Ager,Age
0,1,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
1,2,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
2,3,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
3,4,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
4,5,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,471,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1
1418,472,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1
1419,473,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1
1420,474,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1


#### Rename columns

In [55]:
aging_rename=aging_melt.rename(columns={"Sample ID": "id", "Genus":"genus", "Species":"species", "Species Code": "code", "Site":"site", "River":"river", "Basin":"basin", "Lat":"lat", "Long":"long", "Date Collected":"date", "Status Upon Collection":"status", "Ager":"ager", "Age":"age"})
aging_rename

,id,genus,species,code,site,river,basin,lat,long,date,status,ager,age
0,1,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
1,2,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
2,3,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
3,4,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
4,5,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z age,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,471,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1
1418,472,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1
1419,473,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1
1420,474,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B age,1


#### Rename ager values 'Z age', 'L age' & 'B age' to 'Z', 'L' & 'B'

In [56]:
aging_rename['ager']=aging_rename['ager'].replace({'Z age': 'Z','L age':'L','B age':'B'})
aging_rename

,id,genus,species,code,site,river,basin,lat,long,date,status,ager,age
0,1,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
1,2,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
2,3,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
3,4,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
4,5,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,471,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B,1
1418,472,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B,1
1419,473,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B,1
1420,474,Lampsilis,teres,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,Alive,B,1


#### Rename dataframe to df (easier when making figures)

In [58]:
df = aging_rename
df.head()

,id,genus,species,code,site,river,basin,lat,long,date,status,ager,age
0,1,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
1,2,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
2,3,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
3,4,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
4,5,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,


### Download dependencies for figures 

In [29]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
import plotly.graph_objects as go


In [60]:
df.head()

,id,genus,species,code,site,river,basin,lat,long,date,status,ager,age
0,1,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
1,2,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
2,3,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
3,4,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,
4,5,Amblema,plicata,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,Alive,Z,


In [65]:
df['site'].value_counts()

Altair, TX       339
Gonzales, TX     303
Victoria, TX     300
Bay City, TX     201
Simonton, TX     150
Bellville, TX     72
Navasota, TX      57
Name: site, dtype: int64

In [77]:
#At work, I am treated with respect.
#defining list_updatemenus
list_updatemenus = [{'label': 'ager',
                     'method': 'update',
                     'args': [{'visible': [True, True, True, False, False, False, False, False, False, False, False, False]}, {'title': ''}]},
                    {'label': 'species',
                     'method': 'update',
                     'args': [{'visible': [False, False, False, True, True, False, False, False, False, False, False, False]}, {'title': ''}]},
                    {'label': 'site',
                     'method': 'update',
                     'args': [{'visible': [False, False, False, False, False, True, True, True, True, True, True, True]}, {'title': ''}]},   
                   ]
#defining graph objects
x_ager1 = df[df['ager']=="Z"]["age"]
x_ager2 = df[df['ager']=="L"]["age"]
x_ager3 = df[df['ager']=="B"]["age"]
x_teres = df[df['species']=="teres"]["age"]          
x_plicata = df[df['species']=="plicata"]["age"]
x_site1 = df[df['site']=="Altair, TX"]["age"]
x_site2 = df[df['site']=="Gonzales, TX "]["age"]
x_site3 = df[df['site']=="Victoria, TX"]["age"]
x_site4 = df[df['site']=="Bay City, TX"]["age"]
x_site5 = df[df['site']=="Simonton, TX"]["age"]
x_site6 = df[df['site']=="Bellville, TX"]["age"]
x_site7 = df[df['site']=="Navasota, TX"]["age"]
                 
#defining data
data=[go.Histogram(x=x_ager1,name='Z',opacity = .5),
      go.Histogram(x=x_ager2,name='L',opacity = .5),
      go.Histogram(x=x_ager3,name='B',opacity = .5),
      go.Histogram(x=x_teres,name='Lampsilis teres',opacity = .5),
      go.Histogram(x=x_plicata,name='Amblema plicata',opacity = .5),
      go.Histogram(x=x_site1,name='Altair',opacity = .5),
      go.Histogram(x=x_site2,name='Gonzales', opacity = .5),
      go.Histogram(x=x_site3,name='Victoria',opacity = .5),
      go.Histogram(x=x_site4,name='Bay City',opacity = .5),
      go.Histogram(x=x_site5,name='Simonton',opacity = .5),
      go.Histogram(x=x_site6,name='Bellville',opacity = .5),
      go.Histogram(x=x_site7,name='Navasota',opacity = .5),
     
     ]


#defining layout
layout=go.Layout(title='',updatemenus=list([dict(buttons= list_updatemenus,type = 'buttons')]),barmode='stack')

#defining layout and plotting
fig = go.Figure(data,layout)
fig.show()